In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_03 import *

## DataBunch/Learner

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=4799)

In [3]:
x_train,y_train,x_valid,y_valid = get_data()
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)
nh,bs = 50,64
c = y_train.max().item()+1
loss_func = F.cross_entropy
train_ds

Factor out the connected pieces of info out of the fit() argument list

`fit(epochs, model, loss_func, opt, train_dl, valid_dl)`

Let's replace it with something that looks like this:

`fit(1, learn)`

This will allow us to tweak what's happening inside the training loop in other places of the code because the `Learner` object will be mutable, so changing any of its attribute elsewhere will be seen in our training loop.

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=5363)

In [4]:
#export
class DataBunch():
    def __init__(self, train_dl, valid_dl, c=None):
        self.train_dl,self.valid_dl,self.c = train_dl,valid_dl,c # c is final number of acitvations
        
    @property
    def train_ds(self): return self.train_dl.dataset
        
    @property
    def valid_ds(self): return self.valid_dl.dataset

In [5]:
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)
data.train_dl.dataset
get_dls(train_ds, valid_ds, bs)

(<torch.utils.data.dataloader.DataLoader at 0x7fdcf51aa6a0>,
 <torch.utils.data.dataloader.DataLoader at 0x7fdcf51aa7b8>)

In [48]:
#export
def get_model(data, lr=0.5, nh=50):
    m = data.train_ds.x.shape[1]
    model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,data.c))
    return model, optim.SGD(model.parameters(), lr=lr)

class Learner():
    def __init__(self, model, opt, loss_func, data):
        self.model,self.opt,self.loss_func,self.data = model,opt,loss_func,data

NameError: name 'model' is not defined

In [7]:
learn = Learner(*get_model(data), loss_func, data)


In [8]:
def fit(epochs, learn):
    for epoch in range(epochs):
        learn.model.train()
        for xb,yb in learn.data.train_dl:
            loss = learn.loss_func(learn.model(xb), yb)
            loss.backward()
            learn.opt.step()
            learn.opt.zero_grad()

        learn.model.eval()
        with torch.no_grad():
            tot_loss,tot_acc = 0.,0.
            for xb,yb in learn.data.valid_dl:
                pred = learn.model(xb)
                tot_loss += learn.loss_func(pred, yb)
                tot_acc  += accuracy (pred,yb)
        nv = len(learn.data.valid_dl)
        print(epoch, tot_loss/nv, tot_acc/nv)
    return tot_loss/nv, tot_acc/nv

In [9]:
loss,acc = fit(1, learn)

0 tensor(0.1524) tensor(0.9574)


## CallbackHandler

This was our training loop (without validation) from the previous notebook, with the inner loop contents factored out:

```python
def one_batch(xb,yb):
    pred = model(xb)
    loss = loss_func(pred, yb)
    loss.backward()
    opt.step()
    opt.zero_grad()
    
def fit():
    for epoch in range(epochs):
        for b in train_dl: one_batch(*b)
```

Add callbacks so we can remove complexity from loop, and make it flexible:

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=5628)

In [10]:
def one_batch(xb, yb, cb): # xb yb are taining data performs fitting of a single batch
    #(dir(cb))
    if not cb.begin_batch(xb,yb): return
    loss = cb.learn.loss_func(cb.learn.model(xb), yb)
    if not cb.after_loss(loss): return
    loss.backward()
    if cb.after_backward(): cb.learn.opt.step()
    if cb.after_step(): cb.learn.opt.zero_grad()

def all_batches(dl, cb): # loops though each batch
    print("In all batches function: " + str(dir(cb)))
    for xb,yb in dl:
        one_batch(xb, yb, cb)
        if cb.do_stop(): return

def fit(epochs, learn, cb): # loops through epochs
    print("Fit function: " +str(dir(cb)))
    if not cb.begin_fit(learn): return
    for epoch in range(epochs):
        if not cb.begin_epoch(epoch): continue
        all_batches(learn.data.train_dl, cb) # learn.data is the learner, containing data and model
        
        if cb.begin_validate():
            with torch.no_grad(): all_batches(learn.data.valid_dl, cb)
        if cb.do_stop() or not cb.after_epoch(): break
    cb.after_fit()

In [11]:
class Callback():
    def begin_fit(self, learn): # add the learner to the call back
        self.learn = learn
        print(learn)
        return True
    def after_fit(self): return True
    def begin_epoch(self, epoch):
        self.epoch=epoch
        return True
    def begin_validate(self): 
        print("In the callback begin_validate")
        return True
    def after_epoch(self): return True
    def begin_batch(self, xb, yb):
        self.xb,self.yb = xb,yb
        return True
    def after_loss(self, loss):
        self.loss = loss
        return True
    def after_backward(self): return True
    def after_step(self): return True

In [12]:
class CallbackHandler():
    def __init__(self,cbs=None):
        self.cbs = cbs if cbs else [] 

    def begin_fit(self, learn):
        self.learn,self.in_train = learn,True
        learn.stop = False
        res = True
        #print("In the call back handler begin_fit method")
        for cb in self.cbs: res = res and cb.begin_fit(learn)
        return res

    def after_fit(self):
        res = not self.in_train
        print("In after fit call back handler")
        for cb in self.cbs: res = res and cb.after_fit()
        return res
    
    def begin_epoch(self, epoch):
        self.learn.model.train()
        self.in_train=True
        res = True
        for cb in self.cbs: res = res and cb.begin_epoch(epoch)
        return res

    def begin_validate(self):
        self.learn.model.eval()
        self.in_train=False
        res = True
        print("In the call back hander begin_validate")
        for cb in self.cbs: res = res and cb.begin_validate()
        return res

    def after_epoch(self):
        res = True
        for cb in self.cbs: res = res and cb.after_epoch()
        return res
    
    def begin_batch(self, xb, yb):
        res = True
        for cb in self.cbs: res = res and cb.begin_batch(xb, yb)
        return res

    def after_loss(self, loss):
        res = self.in_train
        for cb in self.cbs: res = res and cb.after_loss(loss)
        return res

    def after_backward(self):
        res = True
        for cb in self.cbs: res = res and cb.after_backward()
        return res

    def after_step(self):
        res = True
        for cb in self.cbs: res = res and cb.after_step()
        return res
    
    def do_stop(self):
        try:     return self.learn.stop
        finally: self.learn.stop = False

In [13]:
class TestCallback(Callback): # a call back is a class inherited from Callback class
    def begin_fit(self,learn): # only amend the callbacks where we add them
        super().begin_fit(learn)
        self.n_iters = 0
        return True
        
    def after_step(self):
        self.n_iters += 1
        print(self.n_iters)
        if self.n_iters>=10: self.learn.stop = True
        return True

In [14]:
fit(1, learn, cb=CallbackHandler([TestCallback()]))

Fit function: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'after_backward', 'after_epoch', 'after_fit', 'after_loss', 'after_step', 'begin_batch', 'begin_epoch', 'begin_fit', 'begin_validate', 'cbs', 'do_stop']
In all batches function: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'after_backward', 'after_epoch', 'after_fit', 'after_loss', 'after_step', 'begin_batch', 'begin_epoch', 'begin_fit', '

In [15]:
#%debug

This is roughly how fastai does it now (except that the handler can also change and return `xb`, `yb`, and `loss`). But let's see if we can make things simpler and more flexible, so that a single class has access to everything and can change anything at any time. The fact that we're passing `cb` to so many functions is a strong hint they should all be in the same class!

## Runner

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=5811)

In [16]:
#export
import re

_camel_re1 = re.compile('(.)([A-Z][a-z]+)')
_camel_re2 = re.compile('([a-z0-9])([A-Z])')
def camel2snake(name):
    s1 = re.sub(_camel_re1, r'\1_\2', name)
    return re.sub(_camel_re2, r'\1_\2', s1).lower()

class Callback():
    _order=0
    def set_runner(self, run): self.run=run
    def __getattr__(self, k): return getattr(self.run, k)
    @property
    def name(self):
        name = re.sub(r'Callback$', '', self.__class__.__name__)
        return camel2snake(name or 'callback')

This first callback is reponsible to switch the model back and forth in training or validation mode, as well as maintaining a count of the iterations, or the percentage of iterations ellapsed in the epoch.

In [17]:
#export
class TrainEvalCallback(Callback):
    def begin_fit(self):
        self.run.n_epochs=0.
        self.run.n_iter=0
    
    def after_batch(self):
        if not self.in_train: return
        self.run.n_epochs += 1./self.iters
        self.run.n_iter   += 1
        
    def begin_epoch(self):
        self.run.n_epochs=self.epoch
        self.model.train()
        self.run.in_train=True

    def begin_validate(self):
        self.model.eval()
        self.run.in_train=False

We'll also re-create our TestCallback

In [18]:
class TestCallback(Callback):
    def after_step(self):
        if self.train_eval.n_iters>=10: return True

In [19]:
cbname = 'TrainEvalCallback'
camel2snake(cbname)

'train_eval_callback'

In [20]:
TrainEvalCallback().name

'train_eval'

In [21]:
#export
from typing import *

def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o]

In [22]:
#export
class Runner():
    def __init__(self, cbs=None, cb_funcs=None):
        cbs = listify(cbs)
        #print(cbs)
        #print(cb_funcs)
        for cbf in listify(cb_funcs):
            cb = cbf()
            setattr(self, cb.name, cb)
            cbs.append(cb)
        self.stop,self.cbs = False,[TrainEvalCallback()]+cbs
       # print(self.cbs)

    @property
    def opt(self):       return self.learn.opt
    @property
    def model(self):     return self.learn.model
    @property
    def loss_func(self): return self.learn.loss_func
    @property
    def data(self):      return self.learn.data

    def one_batch(self, xb, yb):
        self.xb,self.yb = xb,yb
        if self('begin_batch'): return # if call back function returns TRUE then return from method
        self.pred = self.model(self.xb)
        if self('after_pred'): return
        self.loss = self.loss_func(self.pred, self.yb)
        if self('after_loss') or not self.in_train: return # don't run the backward pass 
        self.loss.backward()
        if self('after_backward'): return
        self.opt.step()
        if self('after_step'): return
        self.opt.zero_grad()

    def all_batches(self, dl):
        self.iters = len(dl)
        for xb,yb in dl:
            if self.stop: break
            self.one_batch(xb, yb)
            self('after_batch')
        self.stop=False

    def fit(self, epochs, learn):
        self.epochs,self.learn = epochs,learn

        try:
            for cb in self.cbs: cb.set_runner(self)
            if self('begin_fit'): return
            for epoch in range(epochs):
                self.epoch = epoch
                if not self('begin_epoch'): self.all_batches(self.data.train_dl)

                with torch.no_grad(): 
                    if not self('begin_validate'): self.all_batches(self.data.valid_dl)
                if self('after_epoch'): break
            
        finally:
            self('after_fit')
            self.learn = None

    def __call__(self, cb_name): # calls the call back
        #print(cb_name)
        for cb in sorted(self.cbs, key=lambda x: x._order): # sorts call backs by order
            f = getattr(cb, cb_name, None) # if the cb has a cb_name function then get it
            if f and f(): return True
        return False
    

Third callback: how to compute metrics.

In [23]:
#export
class AvgStats():
    def __init__(self, metrics, in_train): 
        self.metrics,self.in_train = listify(metrics),in_train
        print("Metrics:")
        print(metrics)
    def reset(self):
        self.tot_loss,self.count = 0.,0
        self.tot_mets = [0.] * len(self.metrics)
        
    @property
    def all_stats(self): return [self.tot_loss.item()] + self.tot_mets
    @property
    def avg_stats(self): return [o/self.count for o in self.all_stats]
    
    def __repr__(self):
        if not self.count: return ""
        return f"{'train' if self.in_train else 'valid'}: {self.avg_stats}"

    def accumulate(self, run):
        bn = run.xb.shape[0]
        self.tot_loss += run.loss * bn
        self.count += bn
        for i,m in enumerate(self.metrics):
            self.tot_mets[i] += m(run.pred, run.yb) * bn # can have any metrics on the predictions and truth 

class AvgStatsCallback(Callback):
    def __init__(self, metrics):
        self.train_stats,self.valid_stats = AvgStats(metrics,True),AvgStats(metrics,False)
        
    def begin_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()
        
    def after_loss(self):
        stats = self.train_stats if self.in_train else self.valid_stats
        with torch.no_grad(): 
            stats.accumulate(self.run) # passes the runner in 
            #print(self.run)
    
    def after_epoch(self):
        print(self.train_stats)
        print(self.valid_stats)

In [24]:
learn = Learner(*get_model(data), loss_func, data)

In [25]:
stats = AvgStatsCallback([accuracy])
run = Runner(cbs=stats)
#run.cbs[1].after_loss

Metrics:
[<function accuracy at 0x7fdd682abf28>]
Metrics:
[<function accuracy at 0x7fdd682abf28>]


In [26]:
run.fit(4, learn)

train: [0.3131054296875, tensor(0.9039)]
valid: [0.3805385986328125, tensor(0.8874)]
train: [0.139987109375, tensor(0.9575)]
valid: [0.800574755859375, tensor(0.8363)]
train: [0.108585205078125, tensor(0.9672)]
valid: [0.11700767822265624, tensor(0.9657)]
train: [0.086562412109375, tensor(0.9733)]
valid: [0.13040943603515626, tensor(0.9626)]


In [27]:
loss,acc = stats.valid_stats.avg_stats
assert acc>0.9
loss,acc

(0.13040943603515626, tensor(0.9626))

In [28]:
#export
from functools import partial

In [29]:
acc_cbf = partial(AvgStatsCallback,accuracy)

In [30]:
run = Runner(cb_funcs=acc_cbf)

Metrics:
<function accuracy at 0x7fdd682abf28>
Metrics:
<function accuracy at 0x7fdd682abf28>


In [31]:
run.fit(1, learn)

train: [0.0753509375, tensor(0.9768)]
valid: [0.24451328125, tensor(0.9341)]


Using Jupyter means we can get tab-completion even for dynamic code like this! :)

In [35]:
run.avg_stats.valid_stats.avg_stats

[0.24451328125, tensor(0.9341)]

## Export

In [33]:
!python notebook2script.py 04_callbacks.ipynb

Converted 04_callbacks.ipynb to exp/nb_04.py


In [41]:
learn.opt.param_groups

[{'params': [Parameter containing:
   tensor([[ 0.0230, -0.0205,  0.0076,  ..., -0.0353,  0.0348,  0.0065],
           [-0.0134,  0.0194,  0.0148,  ..., -0.0109,  0.0058,  0.0190],
           [-0.0323, -0.0012, -0.0277,  ..., -0.0259, -0.0062,  0.0068],
           ...,
           [-0.0046, -0.0174, -0.0084,  ...,  0.0255, -0.0038, -0.0309],
           [ 0.0002,  0.0317,  0.0351,  ...,  0.0155,  0.0164, -0.0183],
           [ 0.0246,  0.0097,  0.0157,  ...,  0.0257,  0.0124, -0.0309]],
          requires_grad=True), Parameter containing:
   tensor([ 0.0724,  0.0529,  0.0251, -0.0446, -0.1368, -0.4361, -0.0056,  0.1303,
           -0.1109,  0.1911, -0.0230,  0.3415, -0.2971,  0.1687, -0.0615,  0.0522,
           -0.1378,  0.0323,  0.0060,  0.2873, -0.2088, -0.2602, -0.1907,  0.0306,
            0.1425,  0.2277, -0.2964,  0.4258,  0.3529,  0.4387, -0.0408,  0.0035,
            0.0250,  0.4011, -0.1606,  0.0820,  0.1604, -0.3670,  0.0606, -0.1339,
            0.3228,  0.2445,  0.1874,  0.3

In [43]:
# Recorder class - 
class Recorder(Callback):
    def begin_fit(self): self.lrs,self.losses = [],[]

    def after_batch(self):
        if not self.in_train: return
        self.lrs.append(self.opt.param_groups[-1]['lr'])
        self.losses.append(self.loss.detach().cpu())        

    def plot_lr  (self): plt.plot(self.lrs)
    def plot_loss(self): plt.plot(self.losses)
        
rcd_cbf = Recorder

run = Runner(cb_funcs=[acc_cbf, Recorder])
run.fit(2, learn)


Metrics:
<function accuracy at 0x7fdd682abf28>
Metrics:
<function accuracy at 0x7fdd682abf28>
train: [0.0655296923828125, tensor(0.9788)]
valid: [0.1013311767578125, tensor(0.9721)]
train: [0.058111953125, tensor(0.9812)]
valid: [0.35156630859375, tensor(0.9118)]


In [47]:
run.learn